## Feature Selection

Multiple Forward Selection

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import f
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
%matplotlib inline

In [2]:
#data = pd.read_csv('winequality-red.csv', sep=';')
data = pd.read_csv('winequality-white.csv', sep=';')

In [3]:
X, y = data.iloc[:,:-1].to_numpy(), data.iloc[:,-1].to_numpy()

In [4]:
scaler = MinMaxScaler(feature_range=(0,10))

In [5]:
num_examples, num_features = X.shape

In [6]:
X = scaler.fit_transform(X)
y = y.reshape((num_examples, 1))

**Normal Equation**:

In [7]:
def normal_equation(X, y):
    '''
    Returns the estimated parameters
    as a (n+1,1) vector
    [n: number of features].
    '''
    return np.linalg.pinv(X.T.dot(X)).dot(X.T).dot(y)

In [8]:
def hypothesis(theta, X):
    '''
    Returns the hypothesis for the LinReg
    theta: (n+1, 1) vector [n: # features]
    X: (m, n+1) matrix [m: # examples]
    '''
    return X.dot(theta)

[Root Mean Squared Error](https://en.wikipedia.org/wiki/Root-mean-square_deviation)

In [9]:
def rmse(predicted, actual):
    err = predicted - actual
    return float(np.sqrt(err.T.dot(err)))

**F-test**:

[F-test to compare regression models](https://en.wikipedia.org/wiki/F-test#Regression_problems)

In [10]:
def f_test(residuals_UR, residuals_R, params_UR, params_R, alpha):
    '''
    Performs an F-test between the unrestricted (UR)
    and restricted (R) model, with the informed
    level of significance (alpha).
    Receives the residuals and number of paramaters
    for each model.
    Returns the F-statistic and F-critical,
    respectively.
    '''
    sample_size = len(residuals_UR)
    # Un-restricted RSS
    rss_UR = residuals_UR.T.dot(residuals_UR)
    # Restricted RSS
    rss_R = residuals_R.T.dot(residuals_R)
    
    num = (rss_R-rss_UR)/(params_UR-params_R)
    den = rss_UR/(sample_size-params_UR)
    f_stat = num/den
    f_crit = f.ppf(1.0-alpha,params_UR-params_R,sample_size-params_UR)
    return f_stat, f_crit

### Feature selection

In [11]:
def feature_selection(X, y, base_features=[0], alpha=0.05):
    '''
    FORWARD SELECTION:
    Evaluates the models with one addition
    (one more feature), and compares
    with the hollow model (only intercept),
    using an F-test with the informed level
    of significance (alpha).
    Returns two lists whose sizes are the number
    of features: the first one contains the results
    for the F-tests (True: null hypothesis rejected;
    i.e., the parameter of the added feature is NOT zero);
    the second one, the RMSE if the corresponding feature
    were added.
    X must already account for the constant term
    (i. e., it must already contain a one-valued first column).
    '''
    _, num_features = X.shape
    other_features = [f for f in range(num_features) if f not in base_features]
    hollow_X = X[:,base_features]
    hollow_theta = normal_equation(hollow_X,y)
    H0_rejected = {0: True}
    rmses = {0: rmse(hypothesis(hollow_theta,hollow_X), y)}
    
    for extension in other_features:
        extended_X = X[:,base_features+[extension]]
        extended_theta = normal_equation(extended_X,y)
        
        # Performing the F-test
        hollow_residuals = hypothesis(hollow_theta,hollow_X) - y
        extended_residuals = hypothesis(extended_theta,extended_X) - y
        f_stat, f_crit = f_test(extended_residuals,hollow_residuals,
                                2,1,alpha)
        
        # Evaluating the statistical significance
        
        H0_rejected[extension] = bool(f_stat > f_crit)
        rmses[extension] = rmse(hypothesis(extended_theta,extended_X),y)

    return H0_rejected, rmses

### Multiple forward selection

In [12]:
can_improve = True
selected_features = [0]

while can_improve:
    H0_rejected, rmses = feature_selection(X,y,selected_features)
    print('H0 rejected: ')
    print(H0_rejected)
    print('\nRMSES: ')
    print(rmses)

    ind_best = min(rmses, key=rmses.get)

    if H0_rejected[ind_best] and ind_best not in selected_features:
        selected_features.append(ind_best)
    else:
        can_improve = False

    print('\nSELECTED FEATURES: ')
    print(selected_features)
    print('\n','-'*50, '\n')

H0 rejected: 
{0: True, 1: True, 2: True, 3: True, 4: True, 5: True, 6: True, 7: True, 8: True, 9: True, 10: True}

RMSES: 
{0: 131.60817366834058, 1: 123.68218471793247, 2: 121.13270580533093, 3: 130.14373948903796, 4: 127.20301327424067, 5: 118.27475848477559, 6: 117.11496384063565, 7: 129.39821606664674, 8: 90.1050664084958, 9: 113.96478312383466, 10: 99.30863384236925}

SELECTED FEATURES: 
[0, 8]

 -------------------------------------------------- 

H0 rejected: 
{0: True, 1: True, 2: True, 3: True, 4: True, 5: True, 6: True, 7: True, 9: True, 10: True}

RMSES: 
{0: 90.1050664084958, 1: 89.43942815680393, 2: 87.45637138932057, 3: 89.21432038616528, 4: 89.71482191800456, 5: 87.29692606726819, 6: 88.95146586107919, 7: 89.99544803864492, 9: 88.14013499096104, 10: 77.75682157390537}

SELECTED FEATURES: 
[0, 8, 10]

 -------------------------------------------------- 

H0 rejected: 
{0: True, 1: True, 2: True, 3: True, 4: True, 5: True, 6: True, 7: True, 9: True}

RMSES: 
{0: 77.756821

### Final model:

**White wine**: 0, 8, 10, 7, 5, 3, 1, 4, 2, 9

**Red wine**: 0, 8, 10, 9, 6, 4, 5